In [ ]:
!pip install tqdm
!conda install -c conda-forge ipywidgets --yes
!pip install Cython

In [ ]:
from paths import *
from torchvision import models
import torchvision
import torch
import torch.nn as nn
import pandas as pd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from Mydataset import *
from matplotlib import pyplot as plt
import pickle

In [ ]:
if not os.path.exists(join(paths["SCRIPTS_PATH"],"vision")):
    !git clone https://github.com/pytorch/vision {join(paths["SCRIPTS_PATH"],"vision")}
if not os.path.exists(join(paths["SCRIPTS_PATH"],"coco")):
    !git clone https://github.com/cocodataset/cocoapi {join(paths["SCRIPTS_PATH"],"coco")}
    !cd Pytorch/scripts/coco/PythonAPI && python3 setup.py build_ext install
    
join(paths["SCRIPTS_PATH"],"vision","references","detection")

In [ ]:
!cp -r Pytorch/scripts/vision/references/detection/* .

In [ ]:
CLASSES=["Dog","Motorcycle", "Van", "Bus", "Bicycle", "Car","Person","Man","Truck"]

# 1. Download pre trained model

In [ ]:
my_faster_rcnn=models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=False)
my_faster_rcnn

**1.1 Changing the last layer to adapt it to our classes**

In [ ]:
num_classes = len(CLASSES)
in_features = my_faster_rcnn.roi_heads.box_predictor.cls_score.in_features
my_faster_rcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
my_faster_rcnn

# 2. see and exmple and test our classes

In [ ]:
df_path=os.path.join(paths['ANNOTATION_TRAIN_PATH'],"annotation.txt")
dataset=Mydataset(df_path)

In [ ]:
class_map=dataset.get_class_map()
a_file = join(paths['ANNOTATION_PATH'],"classes.txt")
import json
with open(a_file, 'w') as convert_file:
    convert_file.write(json.dumps(class_map))


In [ ]:
class_map

In [ ]:
df_path_val=os.path.join(paths['ANNOTATION_VAL_PATH'],"annotation.txt")
dataset_val=Mydataset(df_path_val, 720, 720)

In [ ]:
def collate(x): 
    return tuple(zip(*x))

In [ ]:
data_loader_train = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True,collate_fn=collate)

In [ ]:
images,targets = next(iter(data_loader_train))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
targets

In [ ]:
loss_dict = my_faster_rcnn(images,targets)   # Returns losses and detections
sum(loss for loss in loss_dict.values()).item()

In [ ]:
images,targets = next(iter(data_loader_train))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
my_faster_rcnn.eval()

predictions = my_faster_rcnn(images)   

In [ ]:
predictions

# 3. training our model

In [ ]:

import math
import sys
import time

import torch
import torchvision.models.detection.mask_rcnn
import utils
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"
    
    loss_classifier =[]
    loss_box_reg=[]
    loss_objectness=[]
    loss_rpn_box_reg=[]
    
    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters)

    loss_to_plot=[]
    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            loss_to_plot.append(losses.item())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger,loss_to_plot


In [ ]:
#from engine import train_one_epoch
import utils

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("We will be using ",device)



    

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=3, shuffle=True, num_workers=3,
    collate_fn=collate)


data_loader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=1, shuffle=False, num_workers=3,
    collate_fn=collate)

# move model to the right device
my_faster_rcnn.to(device)

    # construct an optimizer
params = [p for p in my_faster_rcnn.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

# let's train it for 20 epochs
num_epochs = 20
total_loss_to_plot=[]
loss_axis=[]
loss_classifier=[]
loss_box_reg=[]
loss_objectness=[]
loss_rpn_box_reg=[]
for epoch in range(num_epochs):
    # train for one epoch, printing every 100 iterations
    metric_logger,loss=train_one_epoch(my_faster_rcnn, optimizer, data_loader, device, epoch, print_freq=100)
    
    
    total_loss_to_plot.append(loss)
    loss_list = metric_logger.meters.get('loss')
    loss_classifier_list=metric_logger.meters.get('loss_classifier')
    loss_box_reg_list=metric_logger.meters.get('loss_box_reg')
    loss_objectness_list=metric_logger.meters.get('loss_objectness')
    loss_rpn_box_reg_list=metric_logger.meters.get('loss_rpn_box_reg')
    
    loss_axis.append(loss_list.value)
    loss_classifier.append(loss_classifier_list.value)
    loss_box_reg.append(loss_box_reg_list.value)
    loss_objectness.append(loss_objectness_list.value)
    loss_rpn_box_reg.append(loss_rpn_box_reg_list.value)
    lr_scheduler.step()


print("That's it!")

In [ ]:
total_loss_to_plot=np.array(total_loss_to_plot)
mean_of_losses=total_loss_to_plot.mean(axis=1)
fig, axs = plt.subplots(3,figsize=(15, 10))


axs[0].plot(total_loss_to_plot)
axs[0].set_title('total Losses for each epochs')

axs[1].plot(total_loss_to_plot[0])
axs[1].set_title('total Loss during the first epoch')

axs[2].plot(mean_of_losses)
axs[2].set_title('mean of total Loss for all epochs')


# 4. save our model

In [ ]:
torch.save(my_faster_rcnn.state_dict(), join(paths["MODEL_PATH"],"dict_pretrained_faster_rcnn.h"))

In [ ]:
torch.save(my_faster_rcnn,join(paths["MODEL_PATH"],"full_pretrained_faster_rcnn.h"))